In [1]:
import os

In [2]:
from typing import TypedDict, Annotated, List

In [3]:
# from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_core.pydantic_v1 import BaseModel

from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.graph import StateGraph, END

from langchain import hub

/Users/jakubdrdak/VScodeProjects/Factchecker/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
from tavily import TavilyClient

In [5]:
from IPython.display import Markdown, display

In [7]:
LANGUAGE = 'english'
TAVILY_MAX_RESULTS = 5
MAX_QUERIES = 5
TAVILY_API_KEY = os.environ["TAVILY_API_KEY"]

In [ ]:
tavily = TavilyClient(api_key=TAVILY_API_KEY)


In [8]:
class AgentState(TypedDict):
    task: str
    plan: str
    draft: str
    translation: str
    target_language: str
    queries: List[str]
    critique: str
    content: List[str]
    revision_number: int
    max_revisions: int

In [9]:
memory = SqliteSaver.from_conn_string(":memory:")


In [10]:
model = ChatOpenAI(model='gpt-4o-mini', temperature=0, verbose=True)

In [11]:
translation_prompt = hub.pull("haha918301/translator") 

In [25]:
translation_prompt.template

'Translate the following text from {input_language} into {output_language}, with the following requirements:\n① Ensure accuracy first\n② Secondly, it must conform to the described context of the given text\n③ Maintain a balance between literal translation and paraphrasing\n④ For any slang that exists, provide an explanation at the end of the translation\n⑤ Output the results in accordance with certain formats\n  1. Translation of the original text\n  2. Paraphrased parts (list the original input-language parts without translating them into output-language, explain the paraphrased translation, and indicate whether it conforms to Chinese expression habits)\n    •\n    •\n    •\n 3. Existence of slang and fixed expressions\n    •\n    •\n    •\n⑥ The following is the paragraph to be translated:\n{text}'

In [12]:
PLAN_PROMPT = """You are an expert writer specialised on unbiased information veryfication in the context of dezinformation, conspiracy theories and propaganda. \
You are tasked with writing a high level outline of an expository article. \
Write such an outline for the user provided topic. Give an outline of the expository article along with any relevant notes \
or instructions for the sections."""

RESEARCH_PLAN_PROMPT = """You are a researcher charged with providing information that can \
be used when writing the following expository article. Generate a list of search queries that will gather \
any relevant information. Only generate {max_queries} queries max.""".format(max_queries=MAX_QUERIES)

TRANSLATION_PROMPT = translation_prompt

DRAFT_PROMPT = """You are an expository article assistant tasked with writing excellent 5-paragraph expository article.\
Generate the best unbiased expository article possible for the user's request and the initial outline. \
If the user provides critique, respond with a revised version of your previous attempts. \
Utilize only the information below, don't add any new information, make up or gues anything, or change the topic. : 

------

{content}"""

REFLECTION_PROMPT = """You are a teacher grading an essay submission. \
Generate critique and recommendations for the user's submission. \
Provide detailed recommendations, including requests for length, depth, style, etc."""



RESEARCH_CRITIQUE_PROMPT = """You are a researcher charged with providing information that can \
be used when making any requested revisions (as outlined below). \
Generate a list of search queries that will gather any relevant information. Only generate 3 queries max."""

In [13]:
def get_plan(state):

    messages = [
        SystemMessage(content=PLAN_PROMPT), 
        HumanMessage(content=state['task'])
    ]
    response = model.invoke(messages)
    plan = response.content
    
    return plan

def plan_node(state: AgentState):
    
    plan = get_plan(state)
    
    return {'plan': plan}

In [14]:
def get_queries(state: AgentState):
    
    class Queries(BaseModel):
        queries: List[str]
        
    messages = [
        SystemMessage(content=RESEARCH_PLAN_PROMPT), 
        HumanMessage(content=state['task'])
    ]
    queries = model.with_structured_output(Queries).invoke(messages)
    queries = queries.queries

    return queries

def get_content(state, queries):

    content = state['content'] or set()
    for q in queries:
        response = tavily.search(query=q, max_results=TAVILY_MAX_RESULTS)
        for r in response['results']:
            content.add(r['content'])
    
    return content

def research_plan_node(state: AgentState):
    
    queries = get_queries(state)
    content = get_content(state, queries)

    return {"content": content, "queries": queries}

In [15]:
def get_draft(state: AgentState):
    
    content = "<<<<<<>>>>>>".join(state['content'] or [])
    user_message = f"{state['task']}\n\nHere is my plan:\n\n{state['plan']}"
    
    draft_promp = DRAFT_PROMPT.format(content=content)
    messages = [
        SystemMessage(content=draft_promp),
        HumanMessage(content=user_message)
    ]
    
    response = model.invoke(messages)
    draft = response.content
    return draft

def draft_node(state: AgentState):
    
    draft = get_draft(state)
    rev_num = state.get("revision_number", 1) + 1

    return {"draft": draft, "revision_number": rev_num}



In [16]:
def get_translation(state: AgentState):
    translation_promp = TRANSLATION_PROMPT.format(input_language=LANGUAGE,
                                             output_language=state['target_language'],
                                             text=state['draft']
                                             )
    messages = [
        HumanMessage(content=translation_promp)
    ]
    response = model.invoke(messages)
    translation = response.content
    
    return translation

def translate_node(state: AgentState):

    translation = get_translation(state)
    
    return {"translation": translation}

In [17]:
builder = StateGraph(AgentState)
builder.add_node("planner_node", plan_node)
builder.add_node("research_plan_node", research_plan_node)
builder.add_node("draft_node", draft_node)
builder.add_node("translate_node", translate_node)


builder.set_entry_point("planner_node")


In [18]:
builder.add_edge("planner_node", "research_plan_node")
builder.add_edge("research_plan_node", "draft_node")
builder.add_edge("draft_node", "translate_node")


In [19]:
graph = builder.compile(checkpointer=memory)

In [20]:
from IPython.display import Image

# Image(graph.get_graph().draw_png())

In [ ]:
thread_num = 0

In [27]:
thread_num += 1
task = "TODO"

thread = {"configurable": {"thread_id": str(thread_num)}}
params = {
    'task': task,
    'target_language': "czech",
    "max_revisions": 2,
    "revision_number": 1,   
}

for s in graph.stream(params, thread, debug=True):
    print(s)

[0:tasks] Starting step 0 with 1 task:
- __start__ -> {'max_revisions': 2,
 'revision_number': 1,
 'target_language': 'czech',
 'task': 'je věrohodný: Jiří Lexa a Kvantová cesta?'}
[0:writes] Finished step 0 with writes to 4 channels:
- task -> 'je věrohodný: Jiří Lexa a Kvantová cesta?'
- target_language -> 'czech'
- revision_number -> 1
- max_revisions -> 2
[1:tasks] Starting step 1 with 1 task:
- planner_node -> {'content': None,
 'critique': None,
 'draft': None,
 'max_revisions': 2,
 'plan': None,
 'queries': None,
 'revision_number': 1,
 'target_language': 'czech',
 'task': 'je věrohodný: Jiří Lexa a Kvantová cesta?',
 'translation': None}
{'planner_node': {'plan': '**Outline for Expository Article: "Is Jiří Lexa and the Quantum Path Credible?"**\n\n**I. Introduction**\n   - A. Definition of credibility in the context of scientific claims\n   - B. Brief introduction to Jiří Lexa and the concept of the "Quantum Path"\n   - C. Purpose of the article: To evaluate the credibility of 

In [28]:
graph.get_state(thread).values['queries']

['Jiří Lexa Kvantová cesta recenze',
 'Jiří Lexa Kvantová cesta věrohodnost',
 'Jiří Lexa Kvantová cesta fakta',
 'Jiří Lexa Kvantová cesta kritika',
 'Jiří Lexa Kvantová cesta analýza']

In [33]:
graph.get_state(thread).values['content']


['Jmenuji se Jiří Lexa, jsem lektor, terapeut, spisovatel a hudební skladatel. Vedle tohoto YouTube kanálu jsem pro vás vytvořil Kvantovou cestu - web plný ins...',
 'Články. Mgr. Jiří Lexa - Lektor, terapeut a inspirátor. 24. 06. 2024 Osobní rozvoj. Od dětství měl Jiří Lexa různé vize a nepochopitelné zážitky, které si nedokázal vysvětlit a zároveň se s tím nemohl nikomu svěřit, neboť po jednom takovém pokusu byl označen jako dítě s bujnou fantazií. Rád pomáhal svému okolí ...',
 'Jiří Lexa: Tak pokud bych to měl vyprávět celé, tak tady budeme hodně dlouho. Nicméně od dětství jsem měl takové různé vize, takové různé zážitky a život mě dovedl k tomu, že jsem se stal lektorem kvantování, kvantovým terapeutem. ... Kvantová cesta Portál osobního rozvoje, jenž se zaměřuje na témata zdravého ...',
 'Co nás čeká v roce 2019? Přímo v živém vysílání budeme svědky toho, jak se čte z kvantového pole. A co za zprávu nebo informaci je tam uloženo? Terapeut, lek...',
 'V životě prožil několik velmi

In [35]:
display(Markdown(graph.get_state(thread).values['draft']))

# Is Jiří Lexa and the Quantum Path Credible?

## I. Introduction
Credibility in the context of scientific claims refers to the trustworthiness and reliability of the information presented, often determined by the evidence supporting it and the qualifications of the individual making the claims. Jiří Lexa is a lecturer, therapist, writer, and music composer who has introduced the concept of the "Quantum Path," a platform aimed at personal development. This article aims to evaluate the credibility of Jiří Lexa's claims regarding the Quantum Path and its alignment with established scientific principles.

## II. Background Information
Jiří Lexa has a diverse professional background that includes roles as a lecturer and therapist. His journey into the realm of personal development began in childhood, marked by various visions and experiences that he struggled to explain. These experiences led him to explore concepts related to quantum theory, ultimately culminating in the creation of the Quantum Path, a web portal focused on personal growth and self-improvement. The Quantum Path encompasses ideas that suggest individuals can shape their lives according to their dreams and desires, a notion that resonates with certain interpretations of quantum mechanics, though it diverges from traditional scientific discourse.

## III. Analysis of Claims
Jiří Lexa's main claims regarding the Quantum Path center on the idea that individuals possess the ability to create their reality through their thoughts and intentions. This concept draws parallels to the observer effect in quantum mechanics, where the act of observation influences the state of a system. However, a thorough examination of the scientific basis for these claims reveals a lack of empirical support. Existing scientific literature does not substantiate the notion that personal intentions can directly alter physical reality in the way Lexa suggests. Furthermore, there is a notable absence of peer-reviewed studies that validate his assertions, raising questions about their scientific grounding. Expert opinions from credible scientists in the field often highlight the distinction between metaphorical interpretations of quantum mechanics and its rigorous scientific applications.

## IV. Evaluation of Evidence
The evidence presented by Jiří Lexa primarily consists of anecdotal accounts and personal experiences rather than empirical data. While anecdotal evidence can provide insights into individual experiences, it lacks the rigor and reliability of scientific data. A critical assessment of the methodology used in any studies or claims associated with the Quantum Path reveals potential biases, as personal beliefs and experiences may influence interpretations. Additionally, the absence of peer-reviewed research further complicates the evaluation of Lexa's claims, suggesting a need for caution when considering the validity of his assertions.

## V. Public Reception and Impact
Public perception of Jiří Lexa and the Quantum Path varies widely, with some individuals embracing his ideas as transformative while others remain skeptical. Media coverage and discussions on social media platforms reflect a mix of enthusiasm and criticism, often highlighting the allure of personal empowerment through quantum concepts. The implications of accepting or rejecting these claims extend to scientific discourse, as they may contribute to misconceptions about quantum mechanics and its applications. The blending of personal development with scientific terminology can lead to misunderstandings, potentially undermining public understanding of established scientific principles.

## VI. Conclusion
In summary, the credibility of Jiří Lexa and the Quantum Path is questionable when evaluated against established scientific standards. While Lexa's ideas may resonate with individuals seeking personal growth, they lack empirical support and rigorous scientific validation. This underscores the importance of critical thinking and evidence-based evaluation in science. Readers are encouraged to seek credible sources and engage in informed discussions, particularly when navigating the intersection of personal development and scientific claims.

In [29]:
display(Markdown(graph.get_state(thread).values['translation']))

### 1. Translation of the original text
# Je Jiří Lexa a Kvantová Cesta důvěryhodní?

## I. Úvod
Důvěryhodnost v kontextu vědeckých tvrzení se vztahuje na spolehlivost a důvěryhodnost prezentovaných informací, často určovanou důkazy, které je podporují, a kvalifikacemi jednotlivce, který tato tvrzení činí. Jiří Lexa je lektor, terapeut, spisovatel a hudební skladatel, který představil koncept "Kvantové Cesty", platformy zaměřené na osobní rozvoj. Cílem tohoto článku je zhodnotit důvěryhodnost tvrzení Jiřího Lexy ohledně Kvantové Cesty a její shody se zavedenými vědeckými principy.

## II. Základní informace
Jiří Lexa má rozmanité profesní zázemí, které zahrnuje role lektora a terapeuta. Jeho cesta do oblasti osobního rozvoje začala v dětství, kdy byl poznamenán různými vizemi a zkušenostmi, které se mu těžko vysvětlovaly. Tyto zkušenosti ho vedly k prozkoumání konceptů souvisejících s kvantovou teorií, což nakonec vyústilo ve vytvoření Kvantové Cesty, webového portálu zaměřeného na osobní růst a seberozvoj. Kvantová Cesta zahrnuje myšlenky, které naznačují, že jednotlivci mohou formovat své životy podle svých snů a přání, což je pojetí, které rezonuje s určitými interpretacemi kvantové mechaniky, ačkoliv se odchyluje od tradiční vědecké diskuse.

## III. Analýza tvrzení
Hlavní tvrzení Jiřího Lexy ohledně Kvantové Cesty se soustředí na myšlenku, že jednotlivci mají schopnost vytvářet svou realitu prostřednictvím svých myšlenek a záměrů. Tento koncept má paralely s efektem pozorovatele v kvantové mechanice, kde akt pozorování ovlivňuje stav systému. Nicméně důkladné prozkoumání vědeckého základu těchto tvrzení odhaluje nedostatek empirické podpory. Existující vědecká literatura nepodporuje myšlenku, že osobní záměry mohou přímo měnit fyzickou realitu tak, jak Lexa naznačuje. Dále je zde výrazný nedostatek recenzovaných studií, které by ověřovaly jeho tvrzení, což vyvolává otázky o jejich vědeckém zakotvení. Odborné názory důvěryhodných vědců v oboru často zdůrazňují rozdíl mezi metaforickými interpretacemi kvantové mechaniky a jejími rigorózními vědeckými aplikacemi.

## IV. Hodnocení důkazů
Důkazy předložené Jiřím Lexou se primárně skládají z anekdotických výpovědí a osobních zkušeností, nikoli z empirických dat. Ačkoli anekdotické důkazy mohou poskytnout náhledy do individuálních zkušeností, postrádají rigoróznost a spolehlivost vědeckých dat. Kritické zhodnocení metodologie použité v jakýchkoli studiích nebo tvrzeních spojených s Kvantovou Cestou odhaluje potenciální zaujatosti, protože osobní víry a zkušenosti mohou ovlivnit interpretace. Dále absence recenzovaného výzkumu dále komplikuje hodnocení Lexových tvrzení, což naznačuje potřebu opatrnosti při zvažování platnosti jeho tvrzení.

## V. Veřejné přijetí a dopad
Veřejné vnímání Jiřího Lexy a Kvantové Cesty se značně liší, přičemž někteří jednotlivci přijímají jeho myšlenky jako transformační, zatímco jiní zůstávají skeptičtí. Mediální pokrytí a diskuse na sociálních médiích odrážejí směs nadšení a kritiky, často zdůrazňující přitažlivost osobního zmocnění prostřednictvím kvantových konceptů. Důsledky přijetí nebo odmítnutí těchto tvrzení sahají do vědecké diskuse, protože mohou přispět k mylným představám o kvantové mechanice a jejích aplikacích. Míchání osobního rozvoje s vědeckou terminologií může vést k nedorozuměním, což může potenciálně podkopat veřejné porozumění zavedeným vědeckým principům.

## VI. Závěr
Shrnuto, důvěryhodnost Jiřího Lexy a Kvantové Cesty je otázná, když je hodnocena podle zavedených vědeckých standardů. Ačkoli Lexovy myšlenky mohou rezonovat s jednotlivci hledajícími osobní růst, postrádají empirickou podporu a rigorózní vědeckou validaci. To podtrhuje důležitost kritického myšlení a hodnocení založeného na důkazech ve vědě. Čtenáři jsou vyzváni, aby hledali důvěryhodné zdroje a zapojili se do informovaných diskusí, zejména při navigaci na pomezí osobního rozvoje a vědeckých tvrzení.

### 2. Paraphrased parts
• "Credibility in the context of scientific claims refers to the trustworthiness and reliability of the information presented"  
   - Paraphrased as "Důvěryhodnost v kontextu vědeckých tvrzení se vztahuje na spolehlivost a důvěryhodnost prezentovaných informací."  
   - This paraphrase maintains the original meaning while using slightly different phrasing, conforming to Czech expression habits.

• "Jiří Lexa is a lecturer, therapist, writer, and music composer"  
   - Paraphrased as "Jiří Lexa je lektor, terapeut, spisovatel a hudební skladatel."  
   - The paraphrase is a direct translation that fits well within Czech expression habits.

• "The blending of personal development with scientific terminology can lead to misunderstandings"  
   - Paraphrased as "Míchání osobního rozvoje s vědeckou terminologií může vést k nedorozuměním."  
   - This paraphrase captures the essence of the original statement while adhering to Czech language norms.

### 3. Existence of slang and fixed expressions
• "Quantum Path" - This term is a specific concept introduced by Jiří Lexa and does not have a direct slang equivalent in Czech. It is treated as a proper noun in the translation.  
• "Observer effect" - This is a scientific term that may not be widely understood outside of academic contexts, but it is not slang. It is translated literally as "efekt pozorovatele."  
• "Personal empowerment" - This phrase is commonly used in self-help contexts and translates to "osobní zmocnění," which is understood in Czech but may not carry the same connotations as in English.